<a href="https://colab.research.google.com/github/Jubayer934/Pneumonia-Detection-by-Symptoms-X-ray-Report/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ==============================
# 0. SETUP & IMPORTS
# ==============================

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# CORRECT PATH (typo fix)
IMAGE_MODEL_PATH = '/content/drive/MyDrive/ModelTrain/Pneumonia/model/pnumonia_image_model.keras'
TABULAR_MODEL_PATH = '/content/drive/MyDrive/ModelTrain/Pneumonia/model/pneumonia_tabuluer_model.pkl'

In [ ]:
!pip -q install ipywidgets pdfkit joblib scikit-learn pandas numpy matplotlib tensorflow
# pdfkit needs wkhtmltopdf on the VM
!apt-get -qq update && apt-get -qq install -y wkhtmltopdf

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import joblib, json, os, base64, datetime
from google.colab import files
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Selecting previously unselected package libavahi-core7:amd64.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../0-libavahi-core7_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-core7:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libdaemon0:amd64.
Preparing to unpack .../1-libdaemon0_0.14-7.1ubuntu3_amd64.deb ...
Unpacking libdaemon0:amd64 (0.14-7.1ubuntu3) ...
Selecting previously unselected package avahi-daemon.
Preparing to unpack .../2-avahi-daemon_0.8-5ubuntu5.2_amd64.deb ...
Unpacking avahi-daemon (0.8-5ubuntu5.2) ...
Selecting previously unselected package libqt5core5a:amd64.
Preparing to unpack .../3-libqt5core

In [ ]:
# Bangladesh Time
import datetime as dt
from datetime import timezone, timedelta
BDT = timezone(timedelta(hours=6))
now = dt.datetime.now(BDT).strftime("%d %B %Y, %I:%M %p")

# --------------------------------------------------------------
# 2. LOAD TABULAR MODEL (same as your original safe_predict)
# --------------------------------------------------------------

In [ ]:
TABULAR_SYSTEM = joblib.load(TABULAR_MODEL_PATH)
tab_model      = TABULAR_SYSTEM['model']
cat_encoder    = TABULAR_SYSTEM['cat_encoder']
scaler         = TABULAR_SYSTEM['scaler']
cat_cols       = TABULAR_SYSTEM['cat_cols']
num_cols       = TABULAR_SYSTEM['num_cols']
tab_features   = TABULAR_SYSTEM['feature_names']

def safe_predict_tabular(patient_dict):
    df = pd.DataFrame([patient_dict])
    df.replace(['', '-', 'None', None], np.nan, inplace=True)

    # ---- categorical ----
    for c in cat_cols:
        df[c] = df[c].fillna('Unknown')

    # ---- numerical ----
    means = dict(zip(num_cols, scaler.mean_))
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(means.get(c, 0))

    # ---- encode & scale ----
    cat_enc = pd.DataFrame(
        cat_encoder.transform(df[cat_cols]),
        columns=cat_encoder.get_feature_names_out(cat_cols),
        index=df.index
    )
    num_sc = pd.DataFrame(
        scaler.transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )
    X = pd.concat([num_sc, cat_enc], axis=1)
    X = X.reindex(columns=tab_features, fill_value=0)

    prob = tab_model.predict_proba(X)[0]
    pred = tab_model.predict(X)[0]
    conf = prob[pred] * 100
    label = "Pneumonia" if pred == 1 else "Normal"
    return label, conf

# --------------------------------------------------------------
# 3. LOAD IMAGE MODEL
# --------------------------------------------------------------

In [ ]:
IMG_MODEL_PATH = "/content/drive/MyDrive/ModelTrain/Pneumonia/model/pnumonia_image_model.keras"
img_model = tf.keras.models.load_model(IMG_MODEL_PATH)
IMG_SIZE = (224, 224)

def preprocess_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0) / 255.0
    return arr

def predict_image(arr):
    prob = img_model.predict(arr, verbose=0)[0][0]          # sigmoid output
    pred = int(round(prob))
    label = "PNEUMONIA" if pred == 1 else "NORMAL"
    conf_p = prob * 100
    conf_n = (1 - prob) * 100
    return label, conf_p, conf_n

# --------------------------------------------------------------
# 4. WIDGETS – CLINICAL + IMAGE UPLOAD
# --------------------------------------------------------------

In [ ]:
style = {'description_width': 'initial'}
layout = widgets.Layout(width='500px')

clinical_inputs = {
    'Gender'             : widgets.Dropdown(options=['M','F'], value='M', description='Gender:', style=style, layout=layout),
    'Age'                : widgets.Text(placeholder='e.g. 65', description='Age:', style=style, layout=layout),
    'Cough'              : widgets.Dropdown(options=['Bloody','Dry','Wet','None','Unknown'], description='Cough:', style=style, layout=layout),
    'Fever'              : widgets.Dropdown(options=['High','Low','Moderate','None','Unknown'], description='Fever:', style=style, layout=layout),
    'Shortness_of_breath': widgets.Dropdown(options=['Severe','Moderate','Mild','None','Unknown'], description='Shortness of Breath:', style=style, layout=layout),
    'Chest_pain'         : widgets.Dropdown(options=['Severe','Moderate','Mild','None','Unknown'], description='Chest Pain:', style=style, layout=layout),
    'Fatigue'            : widgets.Dropdown(options=['Severe','Moderate','Mild','None','Unknown'], description='Fatigue:', style=style, layout=layout),
    'Confusion'          : widgets.Dropdown(options=['Yes','No','Unknown'], description='Confusion:', style=style, layout=layout),
    'Oxygen_saturation'  : widgets.Text(placeholder='e.g. 92', description='O2 Sat (%):', style=style, layout=layout),
    'Crackles'           : widgets.Dropdown(options=['Yes','No'], value='No', description='Crackles:', style=style, layout=layout),
    'Sputum_color'       : widgets.Dropdown(options=['Bloody','Green','Yellow','Clear','Unknown'], description='Sputum Color:', style=style, layout=layout),
    'Temperature'        : widgets.Text(placeholder='e.g. 39.5', description='Temp (°C):', style=style, layout=layout),
}

uploader = widgets.FileUpload(
    accept='image/*', multiple=False,
    description='Upload X-ray', style={'button_color':'#4CAF50'}
)

# --------------------------------------------------------------
# 5. PREDICTION + PDF LOGIC
# --------------------------------------------------------------

In [ ]:
out_box   = widgets.Output()
btn       = widgets.Button(description="Predict & Export PDF", button_style='primary',
                           layout=widgets.Layout(width='300px', margin='20px 0'))

def generate_html_report(patient, tab_res, img_res, img_b64, current_time):
    """Return a full HTML string that will be converted to PDF."""
    tab_label, tab_conf = tab_res
    img_label, img_conf_p, img_conf_n = img_res

    # ---- decide final verdict (simple average of confidences) ----
    final_conf_p = (tab_conf if tab_label == "Pneumonia" else 100 - tab_conf) + img_conf_p
    final_conf_p /= 2
    final_label = "PNEUMONIA" if final_conf_p >= 50 else "NORMAL"

    rows = ""
    for k, v in patient.items():
        rows += f"<tr><td><b>{k.replace('_',' ')}</b></td><td>{v}</td></tr>"

    html = f"""
    <html><head>
    <style>
      body {{font-family: Arial; margin:40px;}}
      table {{width:100%; border-collapse:collapse; margin:20px 0;}}
      th, td {{border:1px solid #ccc; padding:8px; text-align:left;}}
      th {{background:#f0f0f0;}}
      .header {{text-align:center;}}
      .img {{text-align:center; margin:20px 0;}}
      .conf {{font-size:1.2em; margin:10px 0;}}
    </style>
    </head><body>
    <div class="header"><h1>Pneumonia Screening Report</h1>
    <p><i>Generated on {current_time}</i></p></div>

    <h2>Patient Clinical Data</h2>
    <table>{rows}</table>

    <h2>Tabular Model (Clinical)</h2>
    <p class="conf"><b>Prediction:</b> {tab_label} – <b>Confidence:</b> {tab_conf:.2f}%</p>

    <h2>Image Model (Chest X-ray)</h2>
    <p class="conf"><b>Prediction:</b> {img_label} – <b>Pneumonia:</b> {img_conf_p:.2f}% | <b>Normal:</b> {img_conf_n:.2f}%</p>

    <h2>Combined Verdict</h2>
    <p class="conf"><b>Final:</b> <span style="color:#d9534f;">{final_label}</span>
       – <b>Average Pneumonia Confidence:</b> {final_conf_p:.2f}%</p>

    <div class="img"><img src="data:image/png;base64,{img_b64}" width="400"/></div>
    </body></html>
    """
    return html

In [ ]:
def on_predict_click(b):
    with out_box:
        clear_output()
        # ---------- 1. Gather clinical dict ----------
        patient = {}
        for name, w in clinical_inputs.items():
            val = w.value
            if isinstance(w, widgets.Text):
                val = val.strip() or None
            patient[name] = val

        # ---------- 2. Check image ----------
        if not uploader.value:
            print("Please upload a chest X-ray image.")
            return
        uploaded_file = list(uploader.value.values())[0]
        img_bytes = uploaded_file['content']
        img_path  = "/tmp/uploaded_xray.png"
        with open(img_path, "wb") as f:
            f.write(img_bytes)

        # ---------- 3. Run models ----------
        tab_label, tab_conf = safe_predict_tabular(patient)

        img_arr = preprocess_image(img_path)
        img_label, img_conf_p, img_conf_n = predict_image(img_arr)

        # ---------- 4. Encode image for PDF ----------
        with open(img_path, "rb") as f:
            img_b64 = base64.b64encode(f.read()).decode()

        # ---------- 5. Build PDF ----------
        html_content = generate_html_report(patient, (tab_label, tab_conf),
                                            (img_label, img_conf_p, img_conf_n), img_b64, now)

        pdf_path = "/tmp/pneumonia_report.pdf"
        import pdfkit
        pdfkit.from_string(html_content, pdf_path)

        # ---------- 6. Show results ----------
        print("Tabular →", tab_label, f"({tab_conf:.2f}%)")
        print("Image   →", img_label,
              f"(Pneumonia {img_conf_p:.2f}% | Normal {img_conf_n:.2f}%)")
        final_conf = (tab_conf if tab_label=="Pneumonia" else 100-tab_conf) + img_conf_p
        final_conf /= 2
        print(f"\nCombined → {'PNEUMONIA' if final_conf>=50 else 'NORMAL'} "
              f"(avg. pneumonia {final_conf:.2f}%)")

        # ---------- 7. Download button ----------
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
        b64_pdf = base64.b64encode(pdf_bytes).decode()
        href = f'<a href="data:application/pdf;base64,{b64_pdf}" download="Pneumonia_Report.pdf">Download PDF Report</a>'
        display(HTML(href))

btn.on_click(on_predict_click)

# --------------------------------------------------------------
# 6. DISPLAY UI
# --------------------------------------------------------------

In [ ]:
title = HTML("<h2 style='text-align:center;color:#2c3e50;'>Pneumonia Dual-Model Predictor</h2>")
subtitle = HTML("<p style='text-align:center;'>Fill clinical data, upload X-ray, click the button.</p>")

clinical_box = widgets.VBox([widgets.HTML("<b>Clinical Data</b>")] +
                            list(clinical_inputs.values()))
image_box    = widgets.VBox([widgets.HTML("<b>Chest X-ray</b>"), uploader])

display(title, subtitle,
        widgets.HBox([clinical_box, image_box]),
        btn, out_box)

Button(button_style='primary', description='Predict & Export PDF', layout=Layout(margin='20px 0', width='300px…

Output()